# Exercise: Lower bounds on the regret 

Consider the problem of [A/B testing](https://en.wikipedia.org/wiki/A/B_testing) in it's simplest case where you are interested in understanding whether the effect of an intervention in the context of, e.g. medical, advertisement, web page design, etc. was postive or negative. Though instead of randomly splitting your target audience and collecting data in an offline manner, you'd be interested in adaptively reject or accept the intervention. This problem can be modeled as a multi armed bandit (MAB) with 2 arms stated in the following:

- Suppose reward distribution $R_t \mid A_t = 1 \sim \mathcal{N}(0,1)$ and arm 2 follows $R_t \mid A_t = 2 \sim \mathcal{N}(\zeta \Delta,1)$ where $\zeta \in \{-1,1\}$ models the positivity or negativity of the effect and $\Delta$ is the effect size. 

- Assume $\Delta$ is known to the experimenter beforehand though $\zeta$ is **unknown** and the goal is to determine it with minimal regret,
$$ L_{T}(\zeta) = T \mu^* - \sum_{t=1}^{T} \mathbb{E}[ R_{t} ]. $$ 

- Note that deteriming sign of effect size can be reformulated as identifying the best arm in our setting since $\mu^*$ is an (invertible) function of $\zeta$,
$$ 
\mu^* = \mu^*(\zeta) =
\begin{cases}
    \Delta & \zeta = 1 \\
    0 & \zeta = -1 \\
\end{cases}
$$

- Denote the expected reward of the arms as $\nu_{\zeta} = (\mu_1,\mu_2)$ where the dependence on $\zeta$ is implicit. Let $N_{i}(t) = \sum_{j=1}^{t-1} 1\{ A_{j} = i \}$ be the number of times up to time $t$ that arm $i$ was pulled and $\bar{N}_{i}(t)$ to be it's expectation. 

**Goal:** We want to show that **any** action policy suffers bad regret bounds either in positive or negative effect sizes (see [Bubeck et al.](https://arxiv.org/pdf/1302.1611.pdf) for more lower bounds). More precisely, we show lower bounds of the following form:
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{\log(T\Delta^2 / 4)}{2\Delta} $$
This is essentially saying that any action policy would suffer a loss $\bar{L}_T = ({L_{T}(\zeta = +1) + L_{T}(\zeta = -1)})/2 $ logarithmic in time parameter $T$ with a uniform prior on $\zeta$. This is indeed similar to the lower bound in Lai and Robbins since the KL divergence for gaussians is proportional to mean differences, that is the effect size $\Delta$. Proving lower bounds for the general $n$ armed bandits (such as the ones in Lai and Robbins) requires some restriction on action policy. 

## Exploration eploitation tradeoff

The essense of our lower bound is the exploration exlpoitation tradeoff. That is if one takes too much time exploring the arms would suffer a bad regret loss. On the other hand, if we don't explore enough we are doomed in distinguishing the sign of the effect size. We formalize this tradeoff in the following parts.

### Too much exploration could be detrimental

Since the reward of the first arm is already known to the experimenter, exploration would correspond to how often experimenter's strategy pulls the second arm. The following question formalizes the fact that exploring too much can be costly.

**Question 1:** In this question we want to provide a formula for the regret with negative effect. 
- Prove that $L_T(\zeta = -1) = \bar{N}_2(T+1)\Delta. $
- Argue that $ L_T(\zeta = -1) \geq \bar{N}_2(T)\Delta. $

**Answer:** Since $\mathbb{E}[R_{t} \mid A_t] = \mu_{A_t}$, then by linearity of expectation,
$$ 
L_{T}(\zeta) = T\mu^* - \mathbb{E}[\sum_{t=1}^{T} \mu_{A_t}] = T\mu^* - \mathbb{E}[N_{1}(T) \mu_{1} + N_{2}(T) \mu_{2}] = T\mu^* - \mathbb{E}[N_{1}(T)]\mu_{1} + \mathbb{E}[N_{2}(T)]\mu_{2} 
$$ 
The result follows from the fact that $(\mu_1,\mu_2) = (0,\Delta)$ when $zeta = -1$. It is easy to see that $N_{2}(T+1) \geq N_{2}(T)$ therefore $\bar{N}_{2}(T+1) \geq \bar{N}_{2}(T)$.

### Too few exploration may not distinguish the sign of effect size

In this part we want to show that
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{1}{2}\left( L_T(\zeta = +1) + L_{T}(\zeta = -1) \right) \geq \frac{\Delta}{16}\left(
\sum_{t=1}^{T} e^{- \bar{N}_{2}(t) \Delta^2}
\right) $$

Let us parse this lower bound. Suppose you opt a policy where you don't explore after a few rounds. Thus the number of pulls of the second arm is going to stay constant, i.e. $\bar{N}_2(t) = O(1)$ (see wiki for [O notation](https://en.wikipedia.org/wiki/Big_O_notation)), and the right hand side in the lower bound above would be proportional to time parameter $T$ as opposed to logarithmic which we are hoping for. In other words this lower bound is saying that exploration is somewhat necessary if we want our regret to be small.  

**Question 2:** Prove that the average regret is porportional to the sum of probability of picking the incorrect arm at each round,
$$\frac{1}{2}\left( L_T(\zeta = +1) + L_{T}(\zeta = -1) \right) = \frac{\Delta}{2}\left(\sum_{t=1}^{T} 
\mathbb{P}[A_{t} = 1 \mid \zeta = +1] + \mathbb{P}[A_{t} = 2 \mid \zeta = -1]
\right)$$

**Answer:** Observe that
$$\mu^{*}(\zeta = +1) - \mathbb{E}[R_t \mid \zeta = +1] = \mu^{*}(\zeta=+1) - \mu_1\mathbb{P}[A_t = 1 \mid \zeta = +1] - \mu_2 \mathbb{P}[A_t = 1 \mid \zeta = +1]$$
Since $\mu^{*}(\zeta = +1) = \Delta = \mu_2$ we have
$$ \mu^{*}(\zeta = +1) - \mathbb{E}[R_t \mid \zeta = +1] = \Delta \mathbb{P}[A_t = 1 \mid \zeta = +1]$$
where the right hand size is proportional to the probability of picking the sub optimal arm.

What is the optimal action at time $t$? At this point we have access to rewards from both arms. Though it should be clear that rewards from the first arm is uninformative regarding $\zeta$. Thus the decision of which arm should be picked up next should depend on rewards observed from the second arm. It can be shown that the probability of making a mistake (picking the incorrect arm) is minimized if we choose our action based on [likelihood ratio](https://en.wikipedia.org/wiki/Neyman–Pearson_lemma) which would yield the following action policy:

$$ O_t = 
\begin{cases}
 2 & r_{t} > 0 \\
 1 & r_{t} < 0
\end{cases}
$$
where $r_t = \sum_{i = 1}^{t-1}{R_i 1\{A_i = 2\}}$ is the cumulative reward of the second arm up to time $t$. 

**Question 3:** Given $N_{2}(t)$ what distribution does $r_t$ follows?

**Answer:** It follows a normal distribution with mean $N_{2}(t)\zeta \Delta$ and variance $N_{2}(t)$ since sum of normals is also a normal random variable. 

**Question 4:** Using the argument presented above assume that for any action policy we have,
$$ \mathbb{P}[A_{t} = 1 \mid \zeta = +1] + \mathbb{P}[A_{t} = 2 \mid \zeta = -1] \geq \mathbb{P}[O_{t} = 1 \mid \zeta = +1] + \mathbb{P}[O_{t} = 2 \mid \zeta = -1]$$
Given this inequality show that 
$$ \mathbb{P}[O_{t} = 1 \mid \zeta = +1]= \mathbb{P}[O_t = 2 \mid \zeta = -1] \geq \frac{1}{4} e^{- \bar{N}_{2}(t) \Delta^2} $$
by taking the following steps:
- Use the law of total probability to condition on number of times the second arm was pulled $N_{2}(t)$.
- Using Gaussian tail bound $\mathbb{P}[X \geq x] \geq \frac{1}{4}e^{-x^2}$ for $x > 0$ and $X \sim \mathcal{N}(0,1)$ to show a bound on the probability of making a mistake using the optimal action policy ($O_t$) and conclude via [Jensen inequality](https://en.wikipedia.org/wiki/Jensen's_inequality) by taking expectation with respect to randomnes in $N_{2}(t)$.

**Answer:** The probability of making a mistake condition on $\zeta$ is the same by symmetery.
Using the law of total probability we have $ \mathbb{P}[O_t = 2 \mid \zeta = -1] = \mathbb{E}[\mathbb{P}[O_t = 2 \mid \zeta = -1, \, N_{2}(t)]]$ thus it is sufficient to bound the mistake probability given $N_{2}(t)$. Observe that $\mathbb{P}[O_t = 2 \mid \zeta = -1, \, N_{2}(t)] = \mathbb{P}[r_{t} > 0 \mid \zeta = -1, \, N_{2}(t)]$. Since $r_t$ condition on $N_2(t)$ is normally distributed we can make the following bound on mistake probability using the optimal action policy,

$$ \mathbb{P}[r_{t} > 0 \mid \zeta = -1, \, N_{2}(t)] = \mathbb{P}[\frac{r_{t} + N_{2}(t)\Delta}{\sqrt{N_{2}(t)}} >  \sqrt{N_{2}(t)}\Delta \mid \zeta = -1, \, N_{2}(t)] = \mathbb{P}[X > \sqrt{N_{2}(t)} \Delta] \geq \frac{1}{4}e^{-N_{2}(t)\Delta^2} $$

Now using Jensen by taking expectation from the above inequality we conclude that
$$ \mathbb{P}[O_t = 2 \mid \zeta = -1] \geq \frac{1}{4}e^{- \bar{N}_{2}(t) \Delta^2} $$


### Combining the tradeoffs

We now proceed to combine the results obtained above. It is easy to see that 
$$\max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{1}{2} \left( \frac{1}{2}\left( L_T(\zeta = +1) + L_{T}(\zeta = -1) \right) +  L_{T}(\zeta = -1) \right)  $$
Now using the results obtained in Q1 and Q4 we can lower bound the right hand side to obtain,
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{\Delta}{2}( \bar{N}_{2}(T) + \frac{1}{4} \sum_{t=1}^{T} e^{- \bar{N}_{2}(t) \Delta^2} )  $$
This lower bound can be thought of as a **formalization** of the exploration exploitation tradeoff. Notice that having either a very small or large value for $N_{2}(T)$ would result in a large lower bound. Thus we need to find the perfect balance, which we do so in the follwing question.

**Question 5:** Take the following steps:
- (i) Show that
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \min_{0 \leq t \leq T}\frac{\Delta}{2}( t + \frac{T}{4} e^{- t \Delta^2} ) $$
- (ii) By minimizing the term in the right hand side show that 
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{\log(T \Delta^2/4)}{2\Delta} $$

**Answer** Since $\bar{N}_{2}(t)$ is increasing in time $t$ we have $\sum_{t=1}^{T} e^{- \bar{N}_{2}(t) \Delta^2} \geq T e^{-\bar{N}_{2}(T)}$ which implies
$$ \max \left({L_{T}(\zeta = +1), L_{T}(\zeta = -1)}\right) \geq \frac{\Delta}{2}( \bar{N}_{2}(T) + \frac{T}{4} e^{- \bar{N}_{2}(T) \Delta^2} ) $$
Note that $\bar{N}_{2}(T) \leq T$ therefore 
$$ \frac{\Delta}{2}( \bar{N}_{2}(T) + \frac{T}{4} e^{- \bar{N}_{2}(T) \Delta^2} ) \geq \min_{0 \leq t \leq T}\frac{\Delta}{2}( t + \frac{T}{4} e^{- t \Delta^2} ) $$ 
By setting the derivative of the right hand to zero we get $t^*=\frac{\log(T\Delta^2 / 4)}{\Delta^2}$ which gives the desired lower bound in part (ii).